# LLaMA 3

## 加载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("neuralwork/fashion-style-instruct")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'completion', 'context'],
        num_rows: 3193
    })
})

In [3]:
dataset["train"][0]

{'input': "I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.",
 'completion': 'Outfit Combination 1:\n- Top: Fitted white linen shirt\n- Bottom: Slim-fit beige chinos\n- Shoe: Brown leather loafers\n- Accessories: Brown woven belt, aviator sunglasses\n\nOutfit Combination 2:\n- Top: Light blue oxford button-down shirt\n- Bottom: Navy blue tailored trousers\n- Shoe: Tan leather brogues\n- Accessories: Navy blue patterned pocket square, silver wristwatch\n\nOutfit Combination 3:\n- Top: Light gray tailored blazer\n- Bottom: Dark wash denim jeans\n- Shoe: White canvas sneakers\n- Accessories: Black leather belt, silver pendant necklace\n\nOutfit Combination 4:\n- Top: Navy blue polo shirt\n- Bottom: Khaki shorts\n- Shoe: Brown leather sandals\n- Accessories: Navy blue baseball cap, woven brown bracelet\n\nOutfit Combination 5:\n- Top: Light pink dress shirt (with rolled-up sleeves)\n- Bottom: Charcoal gray dr

## 构建指令数据集

In [4]:
def format_instruction(sample) :
    return f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
{sample["input"]}
### Context:
{sample["context" ]}
### Response:
{sample["completion"] }
"""

In [5]:
sample = dataset["train"][0]
print(format_instruction(sample))

You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the evert described in the context to generate 5 self-contained and complete outfit combinations.
### Input:
I'm a tall, athletic man with broad shoulders and a narrow waist. I prefer sharp, tailored suits that highlight my V-shaped torso.
### Context:
I'm going to a nature retreat.
### Response:
Outfit Combination 1:
- Top: Fitted white linen shirt
- Bottom: Slim-fit beige chinos
- Shoe: Brown leather loafers
- Accessories: Brown woven belt, aviator sunglasses

Outfit Combination 2:
- Top: Light blue oxford button-down shirt
- Bottom: Navy blue tailored trousers
- Shoe: Tan leather brogues
- Accessories: Navy blue patterned pocket square, silver wristwatch

Outfit Combination 3:
- Top: Light gray tailored blazer
- Bottom: Dark wash denim jeans
- Shoe: White canvas sneakers
- Accessories: Black leather belt, silver pendant necklace

Outfit Combi

## 定义量化参数

In [6]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 加载预训练模型

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=''
    )

model.config.use_cache=False
model.config.pretraining_tp = 1

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

## 加载分词器

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token='') 
tokenizer.pad_token = tokenizer.eos_token

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:711: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 设置PEFT参数

In [10]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj"
        "up_proj",
        "down_proj"
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05, 
    task_type="CAUSAL_LM",
)

## 构建用于训练的PEFT model

In [11]:
from peft import prepare_model_for_kbit_training, get_peft_model

# prepare model for training

model = prepare_model_for_kbit_training(model)
model = get_peft_model (model, peft_config)

## 打印训练参数数量

In [12]:
def print_trainable_parameters(model):
    # Prints the number of trainable parameters in the model.
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel() 
        if param. requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params/all_param}")
    
# get frozen us trainable model param statistics
print_trainable_parameters(model)

trainable params: 27262976 || all params: 4567863296 || trainable%: 0.5968430803057028


## 设置训练参数

In [13]:
from transformers import TrainingArguments

model_args = TrainingArguments(
    output_dir="llama3-8b-style",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

## 模型微调

In [14]:
from trl import SFTTrainer

max_seq_length=2048

# supervised Fine-Tuning Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=model_args,
)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
trainer.train()

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.170400
20,0.913400
30,0.849300
40,0.810400
50,0.773500
60,0.753200
70,0.722900
80,0.721900
90,0.690500
100,0.678000


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66230d65-592240215ca38f110e718380;4c3f7eb7-a492-4579-85f8-be54c5e0a35c)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Repo model meta-llama/Meta-Llama-3-8B-Instruct is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_re

TrainOutput(global_step=324, training_loss=0.6778162923859962, metrics={'train_runtime': 4917.9854, 'train_samples_per_second': 0.528, 'train_steps_per_second': 0.066, 'total_flos': 2.395336904665989e+17, 'train_loss': 0.6778162923859962, 'epoch': 2.99})

In [16]:
trainer.save_model()

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66231a2f-5596c87945a04d8661598176;c2cca786-de22-4d89-82ab-1ab0d939da97)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Repo model meta-llama/Meta-Llama-3-8B-Instruct is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
